<a href="https://colab.research.google.com/github/jaro19853/dw_matrix_road_signs/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from skimage import color, exposure

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

sns.set()
%load_ext tensorboard

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_signs


In [0]:
train_db = pd.read_pickle('data/train.p')
test_db = pd.read_pickle('data/test.p')

X_train, y_train = train_db['features'], train_db['labels']
X_test, y_test = test_db['features'], test_db['labels']

In [4]:
sign_names = pd.read_csv('data/signnames.csv')
sign_names.head()

,a,b
0,0,Speed limit (20km/h)
1,1,Speed limit (30km/h)
2,2,Speed limit (50km/h)
3,3,Speed limit (60km/h)
4,4,Speed limit (70km/h)


In [0]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:

input_shape = X_train.shape[1:]
cat_num = y_train.shape[1]

In [7]:
def get_cnn_v1(input_shape, cat_num, verbose=False):
    model = Sequential([Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                        Flatten(),
                        Dense(cat_num, activation='softmax')])
    if verbose:
        model.summary()

    return model


cnn_v1 = get_cnn_v1(input_shape, cat_num, True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 43)                2476843   
Total params: 2,478,635
Trainable params: 2,478,635
Non-trainable params: 0
_________________________________________________________________


In [8]:
def train_model(model, X_train, y_train, params_fit=dict()):

    logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    model.fit(X_train, 
              y_train,
              batch_size=params_fit.get('batch_size', 128),
              epochs=params_fit.get('epochs', 5),
              verbose=params_fit.get('verbose', 1),
              validation_data=params_fit.get('validation_data', (X_train, y_train)),
              callbacks=[tensorboard_callback])

    return model

model_trained = train_model(cnn_v1, X_train, y_train)


Epoch 1/5
272/272 [==============================] - 3s 10ms/step - loss: 18.5461 - accuracy: 0.7573 - val_loss: 0.2326 - val_accuracy: 0.9470
Epoch 2/5
272/272 [==============================] - 2s 9ms/step - loss: 0.2024 - accuracy: 0.9551 - val_loss: 0.1295 - val_accuracy: 0.9746
Epoch 3/5
272/272 [==============================] - 2s 9ms/step - loss: 0.1414 - accuracy: 0.9674 - val_loss: 0.1163 - val_accuracy: 0.9718
Epoch 4/5
272/272 [==============================] - 2s 9ms/step - loss: 0.1032 - accuracy: 0.9776 - val_loss: 0.0716 - val_accuracy: 0.9857
Epoch 5/5
272/272 [==============================] - 2s 9ms/step - loss: 0.0774 - accuracy: 0.9825 - val_loss: 0.0726 - val_accuracy: 0.9825


In [0]:
def predict(model, X_test, y_test, scoring=accuracy_score):
    y_test_norm = np.argmax(y_test, axis=1)
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn(input_shape, cat_num):
    model = Sequential([Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                        MaxPool2D(),
                        Dropout(0.3),

                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                        MaxPool2D(),
                        Dropout(0.3),

                        Flatten(),

                        Dense(1024, activation='relu'),
                        Dropout(0.3),

                        Dense(1024, activation='relu'),
                        Dropout(0.3),

                        Dense(cat_num, activation='softmax')])
    return model

In [0]:
def train_and_predict(model, X_train, y_train, X_test, y_test):
    model_trained = train_model(model, X_train, y_train)
    return predict(model_trained, X_test, y_test)

# train_and_predict(get_cnn(input_shape, cat_num), X_train, y_train, X_test, y_test)

In [0]:
def get_model(input_shape, cat_num, params):
    model = Sequential([Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                        MaxPool2D(),
                        Dropout(params['dropout_cnn_0']),

                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                        MaxPool2D(),
                        Dropout(params['dropout_cnn_1']),

                        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                        MaxPool2D(),
                        Dropout(params['dropout_cnn_2']),

                        Flatten(),

                        Dense(1024, activation='relu'),
                        Dropout(params['dropout_dense_0']),

                        Dense(1024, activation='relu'),
                        Dropout(params['dropout_dense_1']),

                        Dense(cat_num, activation='softmax')])
    return model

In [0]:
def func_obj(params):
    model = get_model(input_shape, cat_num, params)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    model.fit(X_train, 
              y_train,
              batch_size=int(params.get('batch_size', 128)),
              epochs=params.get('epochs', 5),
              verbose=params.get('verbose', 0)
    )

    score = model.evaluate(X_test, y_test, verbose=0)
    accuracy = score[1]
    print(f'params={params}')
    print(f'accuracy={accuracy}')

    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_0': hp.uniform('dropout_cnn_0', 0.3, 0.5),
    'dropout_cnn_1': hp.uniform('dropout_cnn_1', 0.3, 0.5),
    'dropout_cnn_2': hp.uniform('dropout_cnn_2', 0.3, 0.5),
    'dropout_dense_0': hp.uniform('dropout_dense_0', 0.3, 0.7),
    'dropout_dense_1': hp.uniform('dropout_dense_1', 0.3, 0.7),
}

In [15]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30, 
    Trials()
)

params={'batch_size': 110.0, 'dropout_cnn_0': 0.3007921336540637, 'dropout_cnn_1': 0.35444705867854315, 'dropout_cnn_2': 0.38609319143499704, 'dropout_dense_0': 0.6230053763675871, 'dropout_dense_1': 0.47158765767245947}
accuracy=0.9369614720344543
params={'batch_size': 150.0, 'dropout_cnn_0': 0.4057864951539986, 'dropout_cnn_1': 0.43620785779772764, 'dropout_cnn_2': 0.42758396053383596, 'dropout_dense_0': 0.6929232885785793, 'dropout_dense_1': 0.46969611591958016}
accuracy=0.8142856955528259
params={'batch_size': 140.0, 'dropout_cnn_0': 0.3045653835686916, 'dropout_cnn_1': 0.43957730527650307, 'dropout_cnn_2': 0.3745179949718148, 'dropout_dense_0': 0.4545352372556494, 'dropout_dense_1': 0.4457426488593522}
accuracy=0.9591836929321289
params={'batch_size': 190.0, 'dropout_cnn_0': 0.4033922126490792, 'dropout_cnn_1': 0.4561534940229903, 'dropout_cnn_2': 0.30395802575094916, 'dropout_dense_0': 0.5196102590506109, 'dropout_dense_1': 0.6789625146619552}
accuracy=0.9179138541221619
params={